In [1]:
import mlflow
import os

# os.environ["AWS_PROFILE"] = 

TRACKING_SERVER_HOST = "ec2-34-229-204-202.compute-1.amazonaws.com" #fill in public DNS of the EC2 Instance Host

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking uri: '{mlflow.get_tracking_uri()}'")

tracking uri: 'http://ec2-34-229-204-202.compute-1.amazonaws.com:5000'


In [3]:


# List all experiments
experiments = mlflow.search_experiments()

# Print experiment names and IDs
for exp in experiments:
    print(f"Name: {exp.name}, ID: {exp.experiment_id}")



Name: my-experiment-1, ID: 1
Name: Default, ID: 0


## Setting up credentials

If encountering credential errors, remember to install awscli and run "aws configure" in terminal. The AWS Access Key Id and AWS Secret Access Key are the keys in the downloaded csv file when the S3 bucket was created.

To install awscli on EC2 ubuntu, follow https://dev.to/abstractmusa/install-aws-cli-command-line-interface-on-ubuntu-1b50

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():
    
    X, y = load_iris(return_X_y=True)
    
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)
    
    lr = LogisticRegression(**params).fit(X,y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y,y_pred))
    
    mlflow.sklearn.log_model(lr, name = "models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")
    
    

2025/07/01 21:21:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-artifacts-remote-101/1/cbfc1c76113446b0bcc255170f8e44ea/artifacts'
🏃 View run lyrical-ram-741 at: http://ec2-34-229-204-202.compute-1.amazonaws.com:5000/#/experiments/1/runs/cbfc1c76113446b0bcc255170f8e44ea
🧪 View experiment at: http://ec2-34-229-204-202.compute-1.amazonaws.com:5000/#/experiments/1


In [5]:
# List all experiments
experiments = mlflow.search_experiments()

# Print experiment names and IDs
for exp in experiments:
    print(f"Name: {exp.name}, ID: {exp.experiment_id}")

Name: my-experiment-1, ID: 1
Name: Default, ID: 0


## Interact with the Model Registry


In [6]:
from mlflow.tracking import MlflowClient

client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [9]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
models = client.search_registered_models()

for model in models:
    print(f"Model Name: {model.name}")


In [11]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="iris_classifier"
)

Successfully registered model 'iris_classifier'.
2025/07/01 21:34:40 WARNING mlflow.tracking._model_registry.fluent: Run with id cbfc1c76113446b0bcc255170f8e44ea has no artifacts at artifact path 'models', registering model based on models:/m-19c027417ae84b9982802d2ec1c2e103 instead
2025/07/01 21:34:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_classifier, version 1
Created version '1' of model 'iris_classifier'.


<ModelVersion: aliases=[], creation_timestamp=1751405680864, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1751405680864, metrics=None, model_id=None, name='iris_classifier', params=None, run_id='cbfc1c76113446b0bcc255170f8e44ea', run_link='', source='models:/m-19c027417ae84b9982802d2ec1c2e103', status='READY', status_message=None, tags={}, user_id='', version='1'>